In [1]:
require(mice)
load("dataex4(1).Rdata")
data <- dataex4

Loading required package: mice
Warning message:
"package 'mice' was built under R version 3.6.3"
Attaching package: 'mice'

The following object is masked from 'package:stats':

    filter

The following objects are masked from 'package:base':

    cbind, rbind



In [2]:
data

y,x1,x2
NA,NA,1.09832134
4.6094758,0.02659437,1.12906729
NA,-1.51655310,1.07485377
1.7182310,-1.36265335,1.94115152
NA,1.17848916,0.82724964
1.8206744,-0.93415132,1.91669412
NA,NA,0.10221299
5.1538614,0.62491779,0.65316976
5.6183299,-0.04572296,0.87393130
1.8078716,NA,2.19901619


In [3]:
# (a)
# impute then transform, https://stefvanbuuren.name/fimd/sec-knowledge.html
#imps <- mice(data, method = ifelse(colnames(data) == "x2", "", "pmm"), m=50, printFlag = FALSE, seed = 1)
# append  data afterward
# base settings
imp0 <- mice(data, maxit=0)
meth <- imp0$method

# passive imputation
meth["x2"] <- ""
meth <- imp0$method
meth
imps <- mice(data, method = meth, maxit = 20, m = 50, seed = 1, printFlag = FALSE)
long <- mice::complete(imps, "long", include = TRUE)
long$x1x2 <- with(long, x1*x2)
imps.itt <- as.mids(long)
fits <- with(imps.itt, exp = lm(y~ x1 + x2 + x1*x2))

y    x1    x2 
"pmm" "pmm"    ""

In [4]:
ests <- pool(fits)
coefs <- summary(ests, conf.int = TRUE)
coefs

term,estimate,std.error,statistic,df,p.value,2.5 %,97.5 %
(Intercept),1.5924203,0.09164948,17.37512,174.0731,0,1.4115331,1.773308
x1,1.4071771,0.09688454,14.52427,207.1716,0,1.2161711,1.598183
x2,1.9688737,0.05243221,37.55085,156.5118,0,1.8653076,2.072440
x1:x2,0.7649632,0.05310998,14.40338,171.1844,0,0.6601284,0.869798


In [13]:
# (b)
data1 <- data
# append new column
data1$x1x2 <- data1$x1*data1$x2

# base settings
imp0 <- mice(data1, maxit=0)
meth <- imp0$method
meth
# passive imputation
meth["x1x2"] <- "~I(x1*x2)"

# modify predictor matrix
pred <- imp0$predictorMatrix

# x1x2 should not be used to predict x1 and x2
pred[c("x1", "x2"), "x1x2"] <- 0

# x2 should be used in the impuation model of x1 though
pred["x2", "x1"] <- 1

# imputation is from left to right, so no need to make sure x1 and x2 are imputed
# before x1x2
imp <- mice(data1, method = meth, predictorMatrix = pred, maxit = 20, m = 50, seed = 1, printFlag = FALSE)

# estimates and CI
# Should regression model be used here anyway?
fits <- with(imp, exp = lm(y~ x1 + x2 + x1*x2))
ests <- pool(fits)
coefs <- summary(ests, conf.int = TRUE)
coefs

y    x1    x2  x1x2 
"pmm" "pmm"    "" "pmm"

term,estimate,std.error,statistic,df,p.value,2.5 %,97.5 %
(Intercept),1.5419850,0.08471307,18.20245,192.3953,0,1.3748995,1.7090706
x1,1.2002017,0.09304429,12.89925,204.9856,0,1.0167552,1.3836483
x2,2.0034521,0.05127264,39.07449,139.2983,0,1.9020789,2.1048253
x1:x2,0.8704996,0.05548953,15.68764,120.4666,0,0.7606385,0.9803607


In [12]:
# (c)
# impute as any other variable after appending a column

data2 <- data
# append new column
data2$x1x2 <- data2$x1*data2$x2

imps <- mice(data2, m=50, printFlag = FALSE, seed = 1)

fits <- with(imps, exp = lm(y~ x1 + x2 + x1*x2))
ests <- pool(fits)
coefs <- summary(ests, conf.int = TRUE)
coefs

term,estimate,std.error,statistic,df,p.value,2.5 %,97.5 %
(Intercept),1.5694097,0.08673122,18.09509,170.91227,0,1.3982074,1.7406121
x1,1.2425974,0.09902039,12.54890,148.81857,0,1.0469299,1.4382650
x2,1.9826115,0.05006505,39.60071,149.03624,0,1.8836825,2.0815405
x1:x2,0.8556482,0.05983244,14.30074,99.14571,0,0.7369298,0.9743666
